In [1]:
import numpy as np
from collections import deque

In [2]:
test = '''
#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#'''

vals = []
for line in test.split('\n')[1:]:
    vals.append(line)
    
vals[:5]

['#.######', '#>>.<^<#', '#.<..<<#', '#>v.><>#', '#<^v^^>#']

In [3]:
vals = []
with open('./input.txt', 'r') as file:
    for line in file:
        vals.append(line.strip())
        
vals[-2:]

['#.v^^>vv.v.>v<v^^.vv^.v.<^^^<>>>.v>><^^<<^<<<^v><^><<vv>v^^^<>v<<v^v.^v^>v.>>>>.v>^<<>^<^v.>v.vv><<v<<^v>.v^<<<<<>^^><vv>#',
 '########################################################################################################################.#']

In [4]:
fakescape = np.array([([v for v in val]) for val in vals])
landscape = np.zeros((5,*fakescape.shape))
for y, fake in enumerate(fakescape):
    for x, f in enumerate(fake):
        if f == "#":
            landscape[0,y,x] = 1
        elif f == ">":
            landscape[1,y,x] = 1
        elif f == "<":
            landscape[2,y,x] = 1
        elif f == "^":
            landscape[3,y,x] = 1
        elif f == "v":
            landscape[4,y,x] = 1
        else:
            continue
landscape.shape

(5, 27, 122)

In [5]:
number_of_moves = 2000
allscape = [landscape,]

for _ in range(number_of_moves):
    newscape = np.zeros_like(landscape)
    for i in range(5):
        y,x = np.argwhere(landscape[i]==1).T
        if i == 1:
            x += 1
            x = np.where(x==landscape.shape[2]-1, 1, x)
        elif i == 2:
            x -=1 
            x = np.where(x==0, landscape.shape[2]-2, x)
        elif i == 3:
            y -= 1
            y = np.where(y==0, landscape.shape[1]-2, y)
        elif i == 4:
            y += 1
            y = np.where(y==landscape.shape[1]-1, 1, y)

        newscape[i,y,x] = 1
    landscape = newscape.copy()
    allscape.append(landscape)
allscape = np.array(allscape).reshape((number_of_moves+1,*landscape.shape))
repeat = np.argwhere((allscape[0]==allscape).all(1).all(1).all(1)).ravel()[1]
repeat

600

In [6]:
def get_unique(situs):
    s = np.array(situations,dtype=object).T
    y,x = s[:2]
    checker = np.array([f"{Y}{X}" for Y,X in zip(y,x)])

    ar, idx = np.unique(checker,return_index=True)
    newsit = []
    for i in idx:
        newsit.append(situations[i])
    return newsit

In [10]:
current=[0,1]

situations = [[current[0],current[1]],]

counter = 0
winning = False
while counter < 2000 and not winning and len(situations)>0:
    counter += 1
    situations = get_unique(situations)
    if counter % 100==0:
        print(counter, len(situations))
        
    new_situations = []
    for y,x in situations:
        landscape = allscape[counter%repeat]

        options = []
        if landscape[:,y,x].sum() == 0:
            options.append([y,x])
        if y-1 >= 0  and landscape[:,y-1, x].sum() == 0:
            options.append([y-1, x])
        if y+1 < landscape.shape[1] and landscape[:,y+1, x].sum() == 0:
            options.append([y+1, x])
        if landscape[:,y, x-1].sum() == 0:
            options.append([y, x-1])
        if landscape[:,y, x+1].sum() == 0:
            options.append([y, x+1])   

        if len(options)>0:
            for o in options:
                Y,X = o
                    
                new_situations.append([Y,X])
                if Y == landscape.shape[1]-1 and X == landscape.shape[2]-2:
                    winning = True                    
                    break

    situations = new_situations
if winning:
    print(f"\nWINNING AFTER {counter} moves")

100 316
200 727

WINNING AFTER 255 moves


In [15]:
phase = 0

goals = []
goals.append([landscape.shape[1]-1 ,landscape.shape[2]-2])
goals.append([0,1])
goals.append([landscape.shape[1]-1 ,landscape.shape[2]-2])

currents = []
currents.append([0,1])
currents.append([landscape.shape[1]-1 ,landscape.shape[2]-2])
currents.append([0,1])

have_been = deque(maxlen=int(repeat))

situations = [[currents[phase][0],currents[phase][1],have_been],]
counter = 0
winning = False
while counter < 20_000 and not winning and len(situations)>0:
    counter += 1
    situations = get_unique(situations)
    if counter % 100==0:
        print(counter, len(situations))
        
    new_situations = []
    for y,x,hb in situations:
        hb.append([y,x])     

        landscape = allscape[counter%repeat]

        options = []
        if landscape[:,y,x].sum() == 0:
            options.append([y,x])
        if y-1 >= 0  and landscape[:,y-1, x].sum() == 0:
            options.append([y-1, x])
        if y+1 < landscape.shape[1] and landscape[:,y+1, x].sum() == 0:
            options.append([y+1, x])
        if landscape[:,y, x-1].sum() == 0:
            options.append([y, x-1])
        if landscape[:,y, x+1].sum() == 0:
            options.append([y, x+1])   

        if len(options)>0:
            for o in options:
                Y,X = o

                if (len(hb)==repeat and hb[0]==[y,x]):
                    continue   
                    
                new_situations.append([Y,X,hb.copy()])
                if Y == goals[phase][0] and X == goals[phase][1]:
                    winning = True                    
                    break                        

    situations = new_situations
    if winning:
        phase += 1
        print(f"Now up to phase {phase+1}")
        if phase == 3:
            winning = True  
            break
        have_been = deque(maxlen=int(repeat))
        situations = [[currents[phase][0],currents[phase][1],have_been],]
        winning = False

        
if winning:
    print(f"\nWINNING AFTER {counter} moves")

100 316
200 727
Now up to phase 2
300 109
400 483
500 856
Now up to phase 3
600 205
700 576
800 874
Now up to phase 4

WINNING AFTER 809 moves
